In [3]:
import gzip
import urllib.request as request
import pandas as pd
import io


In [11]:
def get_codes(dictionary):
      url_dict = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/BulkDownloadListing" + \
          "?sort=1&downfile=dic%2Fen%2F" +\
          dictionary + ".dic"
      with request.urlopen(url_dict) as resp:
                file_content = resp.readlines()
      d={}
      for i in file_content:
            if len(i) > 1:
                  row=i.decode().split('\t')
                  d[row[0]] = row[1].strip()
      return d

In [1]:
def get_df(dataset):
    """ 
    dataset: choose the dataset code from:
https://ec.europa.eu/eurostat/databrowser/explore/all/all_themes?lang=en&display=list&sort=category
    something like EXT_LT_INTROEU27_2020
List of datasets for downloding: 
https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?dir=data&sort=1&sort=2&start=a
Returtn the cleaned dataset with decoded abbreviation

    """

    dataset = dataset.lower()
    url = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/" + \
          "BulkDownloadListing?file=data%2F/" + \
          dataset + ".tsv.gz"

    with request.urlopen(url) as r:
            with gzip.GzipFile(fileobj=r) as data:
                file_content = data.read() #data read as binary from gzip
    # the approach to make the dataframe from the object with read data:
    # https://stackoverflow.com/questions/39213597/convert-text-data-from-requests-object-to-dataframe-with-pandas
    # the eurostat data contains not strict columns:first line with separator ',',the date columns - separator - '\t'
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    #devide one column 'geo\\date'
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    # use only file_content,not df,it is binary, so use decode()
    #file_content has the first row with the code we need to decode into names using function get_code
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [45]:
def clean_df(dataset_file):
    """ 
    use it if you have downloaded file
    https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&dir=dic%2Fen
    """

   
    with gzip.open(dataset_file, 'rb') as f:
        file_content = f.read()
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [56]:
get_df(dataset='EXT_LT_INTROEU27_2020')

,indic_et,sitc06,geo,2021,2020,2019,2018,2017,2016,2015,...,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,Trade balance in million ECU/EURO,"Food, drinks and tobacco",Brazil,38479.5,37179.1,35410.1,30966.5,36346.9,33970.3,35943.7,...,34918.1,30103.7,23201.8,22344.5,20386.4,18696.5,16599.4,13840.3,10837.4,10627.9
1,Trade balance in million ECU/EURO,"Food, drinks and tobacco",Canada,9245.8,6823.2,5241.9,4407.5,3998.2,3596.9,4558.6,...,2105.3,2134.3,2256.2,4836.5,3631.1,3619.7,3950.5,4465.8,3444.9,4900.8
2,Trade balance in million ECU/EURO,"Food, drinks and tobacco",China except Hong Kong,-48287.8,-32935.9,-17135.8,-2188.9,4966.0,8459.1,4676.2,...,14594.0,14381.2,12522.6,12455.9,14038.0,12648.6,10845.5,8341.2,10696.8,10553.2
3,Trade balance in million ECU/EURO,"Food, drinks and tobacco",European Union - 27 countries (from 2020),58440.2,53337.2,45423.5,36346.9,36141.9,32856.1,31877.8,...,17073.6,15169.9,8486.9,9509.4,8680.0,10056.7,7003.1,6233.0,6215.5,6831.7
4,Trade balance in million ECU/EURO,"Food, drinks and tobacco",India,26673.0,20797.5,20362.4,19367.1,16916.2,14486.5,16060.6,...,13255.1,8208.7,5683.6,8690.5,6015.0,5667.3,4957.7,4385.8,4186.5,4955.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,Export/Import ratio,Total - all products,Mexico,1.0,1.1,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.9,1.0,1.0,1.0,1.0,1.0,1.0
572,Export/Import ratio,Total - all products,Russia,1.7,1.5,1.7,1.9,1.5,1.5,1.9,...,1.7,1.7,1.8,1.8,1.8,2.2,2.4,2.4,2.3,2.3
573,Export/Import ratio,Total - all products,Singapore,NaN,1.1,1.1,1.1,1.1,1.2,1.2,...,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.2,1.1
574,Export/Import ratio,Total - all products,United Kingdom,0.7,0.6,0.7,0.7,0.7,0.6,0.7,...,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.8
